In [156]:
# Set environment variables to better visualize DataFrames.
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

In [157]:
# Import relevant packages.
using DataFrames
using CSV
using ProgressBars
using StatsPlots

In [158]:
# Load withdraw and deposit data.
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame);

In [290]:
# Lets load the transactions_of_deposit_addresses, a dataframe that contains all the transactions that include
# a deposit addresses. (transaction.to_address in deposit_addresses or transaction.from_address in deposit_addresses).
# Since we are looking for transactions between two EOA addresses that involve some cryptocurrency amount,
# the transactions where value == 0 were left apart.
transactions_of_deposit_addresses_df =  CSV.read("../data/transactions_of_deposit_addresses_with_value.csv", DataFrame)

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,Int64,String,Int64,Int64,String,String?,Float64,Int64,Int64,Int64,Int64,String?,String?,Float64?,String,Int64,String,Float64?,Float64?,Float64?,Int64
1,11,0x66cf00e45223c2ceb1f1349b17718d9fa0bf4e07b80acc0b3946fb9d24800ad9,1,88,0x6f52ed5e0277b40c642cdca57f4b4aeab4b4ae84,0x467da46112005ac4c30826014ab8a565b4e5038c,9.19105e15,21000,5000000000,3080911,21000,missing,missing,1.0,2020-04-08 17:03:50+00:00,9832773,0x00dc528fb47e4cfcfbea92be1e3cd424d61789becc8bdb6f845cac4991dab80b,missing,missing,missing,5000000000
2,15,0x84a425c0fdb4eb46781098d704cdc72caf1ba6ba6de09f879adf80ddb6800872,64,174,0xce3e7a4af505e7b57b49abc9df8d30d3032fa6ee,0xe592427a0aece92de3edee1f18e0157c05861564,1.0e20,276814,346000000000,8969674,229026,missing,missing,1.0,2021-05-19 12:24:54+00:00,12464798,0x10055ff0d74566de2e57fa9ada31ddadeccfba1f25671e251227e58369900968,missing,missing,missing,346000000000
3,16,0xae7cfae853237463786b43ed97295309df4f7022950edd92f519a3612ccc2a19,19,149,0xb16b9619ccd642241faccc2b71c3f17546db24ba,0x3905021030092cb762d9354c6b3a031898c69f75,9.28e15,21000,36000000000,14958974,21000,missing,missing,1.0,2021-07-29 23:41:09+00:00,12923685,0x4606fd3b65938d2f1226224b19e278d3a446cd7ef84487fe75c93067071d72df,missing,missing,missing,36000000000
4,17,0x20a61d06feadfa8b5d23dbf827b0ab7581378dfeaca5c04293110e498feb77e9,460,81,0xe61123c4e2540ce911fc6aa7207d3310f8e1103b,0xb475d3c483ed134562724d5058ff996482a38000,1.0e18,21000,107000000000,4681317,21000,missing,missing,1.0,2020-10-02 17:09:43+00:00,10977846,0x1f961a1e5174d229075d24a1ac67eac8c3f83cbd9d884e24c6d3d2fab65c3215,missing,missing,missing,107000000000
5,19,0x469229834f02fef6a9457f9fd0555ab347b74fc77e63a2e2e1f9e8a49cc0dbe9,1027,176,0x653d63e4f2d7112a19f5eb993890a3f27b48ada5,0x905b63fff465b9ffbf41dea908ceb12478ec7601,7.76628e17,1200000,41000000000,10972437,1013080,missing,missing,1.0,2020-12-21 05:13:58+00:00,11494738,0x8af6b259e13d11ba4717b1136fd236679c78b28a8bd191c988e439ae41f99a9e,missing,missing,missing,41000000000
6,20,0x801c88bef4641b394cd710e2aa70cb7d3250f2b9f7aa1bfbf22120f3314670e5,469,110,0xe9822f18f2654e606a8dff9d75edd98367e7c0ae,0x784e6df1490ee2b49a293405e443e2f72279cb25,5.0e17,73556,34000000000,8632076,23556,missing,missing,1.0,2020-07-10 05:29:24+00:00,10429893,0x819f8e9a9b7e1d8671e730d05eee41827cfbfcfe0b789b522b31cec3a9f0a9e5,missing,missing,missing,34000000000
7,24,0x10a037a3f9632799eef45f49ecddda5503d64143345af819ad782ff46001ff65,0,166,0x9cda39a650a10b398b88df4fbd9e491f06ec3c03,0x1e815a8188f1b84564577c1c998f7e6b4706b752,3.89929e16,21000,66000000000,9627106,21000,missing,missing,1.0,2020-08-03 11:24:52+00:00,10586529,0x641f4ab0f76794b8d6879415b2c624e96c67fd9ff70e7a6c4796b7e71fbbc50a,missing,missing,missing,66000000000
8,31,0x0c8214a35510593d4bd1e36ace4cfca003ffb61f0cf8d528f42db936db89d8f5,4464,223,0xd90668e94ea31538408a5874db5ccc41cf4cc54b,0xc0c49b81f9996ae22bee60268dfc1e9787e1bb80,9.91179e16,21000,42000000000,11709689,21000,missing,missing,1.0,2021-07-08 16:42:50+00:00,12787831,0x7a995aed0469103e4fee8cb6d1e7260543c2849b7d66c9f6f63eee4687f16814,missing,missing,missing,42000000000
9,34,0xf2948c33bb13ae6be124eb8ba1f41f5d58d687e6a3693f4ba443a5a94f3794bc,4,23,0x4c049d7bb86f06ff2719d5acfd1c16ae9f377d64,0xb164c8adb299a203f23cdb2c70c49f6638550d1c,1.14e15,90000,156000000000,904466,21000,missing,missing,1.0,2021-04-23 00:20:58+00:00,12293232,0x2ea5e342d0d2119e1baad0c1ee6442d17378aacfb1b6edcb13e355f3bcd4e576,missing,missing,missing,156000000000


### function linked_transactions_heuristic
The input DataFrame `transactions_of_deposit_addresses` contain transactions that involve deposit_addresees.
We want to link the input `withdraw_transaction` with a transaction inside the `transactions_of_deposit_addresses`. So, the  `linked_transactions_heuristic` function retuns all the transactions inside the `transactions_of_deposit_addresses` that involve the `withdraw_transaction.recipient_address` and where made before `withdraw_transaction.block_timestamp` 

Important: The input `transactions_of_deposit_addresses` must be sorted  by the column `block_timestamp` 

In [286]:
function linked_transactions_heuristic(transactions_of_deposit_addresses, withdraw_transaction)
    linked_transactions = DataFrame(  withdraw_address = String[], deposit_address = String[], transaction_hash = String[])
    
        for transaction in eachrow(transactions_of_deposit_addresses)
            if transaction.block_timestamp > withdraw_transaction.block_timestamp 
                break
            elseif transaction.from_address == withdraw_transaction.recipient_address
                push!(linked_transactions, [withdraw_transaction.recipient_address, transaction.to_address, transaction.hash ])
            elseif transaction.to_address == withdraw_transaction.recipient_address
                push!(linked_transactions, [withdraw_transaction.recipient_address, transaction.from_address, transaction.hash])
            end
        end

    
    return linked_transactions
end

linked_transactions_heuristic (generic function with 1 method)

In [287]:
# Lets test the function
sorted_withdraw_transactions_df = sort(withdraw_transactions_df, :block_timestamp )
sorted_transactions_of_deposit_addresses = sort(transactions_of_deposit_addresses, :block_timestamp )
linked_transactions_heuristic(sorted_transactions_of_deposit_addresses,sorted_withdraw_transactions_df[80700, :])

,withdraw_address,deposit_address,transaction_hash
,String,String,String
1,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xfea0904acc8df0f3288b6583f60b86c36ea52acd,0x450551e229202bc643b3f45df8721b921869bd43f4b0936ae9cbc00b4a450ca0
2,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xfea0904acc8df0f3288b6583f60b86c36ea52acd,0x0557533e68c813a2bba37ca389233f7618c9327657ccd1cac89ee832dc1be568
3,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0x510059633c4958d43dfbc50b3b1a4ce6b9cb871a,0x1743d7077af28de73217ee384aff0532944b47183981ae8184bbfecde1eba728
4,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xfea0904acc8df0f3288b6583f60b86c36ea52acd,0x6aa263dd363ba99c433a8a4c9e6bfe8a120662029bf1c545fa11644ad04ce8dc
5,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xfea0904acc8df0f3288b6583f60b86c36ea52acd,0xef6fded787e78210c2d00bc83fe7c181fdb7b3d749176efcd4eab3bf60349154
6,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xaaf5feaa9e5694b2b293e67558e2da8ea4b1fb13,0xe359cbc7f65401d6710be820cab203e2c88d00991481033d72268500a6900ba3
7,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xaaf5feaa9e5694b2b293e67558e2da8ea4b1fb13,0xe6c5e52820328d5f00c476138caa9b2aa5cc1727ff7687e5da8757260026efc0
8,0xee439ee079ac05d9d33a6926a16e0c820fb2713a,0xaaf5feaa9e5694b2b293e67558e2da8ea4b1fb13,0xe89565ee3617a2a2f214551c83d1df7008c946e9cea5568b7176a5000cbe21a9


### function apply_linked_transactions_heuristic
Apply the `linked_transactions_heuristic` function to each withdraw of the input `withdraw_transactions_df` DataFrame.
Retuns a DataFrame with the columns: `withdraw_address`, `deposit_address` and `transaction_hash`.
You can read this DataFrame in the following way:
The `withdraw_address` is linked to the  `deposit_address` by the transaction `transaction_hash`

In [139]:
# Auxiliary function
function add_linked_transactions(linked_transactions, new_linked_transactions)
    for transaction in eachrow(new_linked_transactions)
        if transaction.transaction_hash ∉ linked_transactions.transaction_hash
            push!(linked_transactions, transaction)
        end
    end
    return linked_transactions
end
    

add_linked_transactions (generic function with 1 method)

In [140]:
function apply_linked_transactions_heuristic(transactions_of_deposit_addresses, withdraw_transactions_df)
    sorted_withdraw_transactions_df = sort(withdraw_transactions_df, :block_timestamp );
    sorted_transactions_of_deposit_addresses = sort(transactions_of_deposit_addresses, :block_timestamp );
    
    linked_transactions = DataFrame( withdraw_address = String[], deposit_address = String[], transaction_hash = String[])
    for withdraw_transaction in ProgressBar(eachrow(sorted_withdraw_transactions_df), printing_delay= 30)
        linked_transactions = add_linked_transactions(linked_transactions, linked_transactions_heuristic(sorted_transactions_of_deposit_addresses, withdraw_transaction))
    end

    
    return linked_transactions
end

draft_apply_linked_transactions_heuristic (generic function with 1 method)

To optimize the `apply_linked_transactions_heuristic` function we need to reduce the `transactions_of_deposit_addresses`.
To do this we keep the transactions only if they are from a deposit_address to a withdraw_address or viceversa and discard all the other transactions.


In [47]:
deposit_addresses = deposit_transactions_df[!, [:from_address]] |> unique |> Matrix
withdraw_addresses = withdraw_transactions_df[!, [:recipient_address]] |> unique |> Matrix;

In [53]:
@time transactions_of_deposit_addresses_compacted = 
    filter(row -> (row.from_address ∈ deposit_addresses && row.to_address ∈ withdraw_addresses) || (row.from_address ∈ withdraw_addresses && row.to_address ∈ deposit_addresses), dropmissing(transactions_of_deposit_addresses_df, [:from_address, :to_address])

1590.577199 seconds (13.49 M allocations: 226.522 MiB, 0.02% compilation time)


,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,Int64,String,Int64,Int64,String,String,Float64,Int64,Int64,Int64,Int64,String?,String?,Float64?,String,Int64,String,Float64?,Float64?,Float64?,Int64
1,31,0x0c8214a35510593d4bd1e36ace4cfca003ffb61f0cf8d528f42db936db89d8f5,4464,223,0xd90668e94ea31538408a5874db5ccc41cf4cc54b,0xc0c49b81f9996ae22bee60268dfc1e9787e1bb80,9.91179e16,21000,42000000000,11709689,21000,missing,missing,1.0,2021-07-08 16:42:50+00:00,12787831,0x7a995aed0469103e4fee8cb6d1e7260543c2849b7d66c9f6f63eee4687f16814,missing,missing,missing,42000000000
2,391,0x83d74a2f04f31c16b29ae25a0aebd8ca27fcd4929c352276e6fd00871f8854f5,942,70,0x00f0d4220fb4832b3fc40c5054b8cac793d2cfb2,0xd3f1a59920a11dd73902d9e7e67466dd00880b2a,2.479e17,21000,100000000000,6275881,21000,missing,missing,1.0,2021-04-09 22:40:53+00:00,12208407,0x746d657ab077210a35d56530dcdf5479f3a74ce7a9bf8d46ab012a0e58e9068b,missing,missing,missing,100000000000
3,587,0x0311906741a9571d40ecbfea2a57805ac51e8ad4154b5360213f4e48ef261782,48,204,0x05edae0a30b0fdc02a20366fc5afa78725803da4,0x320499df2dad3349ae7c8c7d94c06126e7b29ee1,7.09737e16,21000,64000000000,11392645,21000,missing,missing,1.0,2021-10-18 10:11:44+00:00,13441199,0x4803e1ee9b2de1e266f00f893daaee75300ee7bbb54e87da4ed6fd33bb14900e,missing,missing,missing,64000000000
4,659,0x6a152339cdc599cca70f28bb091602a9a8c78702b66bec28819c6f9001c67c9d,9,133,0xd745c80485ef2792c5fdc02e879f0124756ea065,0x035e8e3fa24b2865c182e52ab1de413f6ec8e5de,5.0e16,21000,57000000000,8293629,21000,missing,missing,1.0,2020-12-21 23:17:40+00:00,11499695,0x7b0d821fdaec06e0dfe527b0a017f5a41a11f4c4f4c038842e53ed3c39684558,missing,missing,missing,57000000000
5,1215,0xa4d23970c676887bed1f52805d1aa4ed3ebbde810d07fd896f1163bc08de57cc,227,86,0x512e07a093aaa20ba288392eadf03838c7a4e522,0x5a9e792143bf2708b4765c144451dca54f559a19,3.9793e17,21000,4000000000,6566317,21000,missing,missing,1.0,2019-04-10 01:42:20+00:00,7537443,0x5eb2687f9c6fd26c61b74ff7c6c1922e2c26aaaa47376c92c565c8e1386daa49,missing,missing,missing,4000000000
6,1492,0xea86b824e3e1d1e03d1d05aed284bdc475955c094d9064d49b6bb4a74105101f,2648,21,0x9dfac9bbe1119da5dcdd7b095db40920cde878ef,0xe5818d70a9b5aed2bfde4e41fbcb07dd80f8fc84,1.5e18,21000,53026600883,1245277,21000,missing,missing,1.0,2021-08-06 13:39:47+00:00,12971724,0xe50f0b7edb5e236918ead7e10f4798324a9c029258bd13683eec6894fa41c5a0,missing,missing,missing,53026600883
7,1614,0x2d864734ba4f298ec241c006eaa60f754f664a0cef49ba88ee64182e947bbb7b,0,14,0x658b266b1dd77114b6b1e39f9ce221d93e1f6d91,0xb887d7cdc2f5cee347ec36abf693a09fde8536f9,6.97842e17,21000,86000000000,1294779,21000,missing,missing,1.0,2021-02-28 05:50:42+00:00,11944069,0xe372ac04210f082941b08ed5d8e9e22583e696affd49437a9390e9aacb5218d5,missing,missing,missing,86000000000
8,1948,0x92399d34eb6059d44c5bf91d75a536824a3ed0d80ef023665597ef4b8b12c222,898,194,0x043b55ec2be886f54ed56908a1b53e6f5b24e92c,0x4af631c5504f7b273845f9b6d5eb39984b8af891,2.32562e16,21000,67000000000,11153521,21000,missing,missing,1.0,2021-01-12 01:09:09+00:00,11637060,0xee583518254d85cf4832671f194310f81aef7fe9974978e0a0fb9e3652104f66,missing,missing,missing,67000000000
9,1986,0xbe22a885d88ba1dd831296364dffd377b8d27945faa451bfb8316663d7d17c1c,44,118,0x40742127fad8597909736bdeba1cd2717315e6ee,0xef07b254bce840acab26094ceb731a313815cb4b,1.55326e17,200000,15000000000,2899325,21000,missing,missing,1.0,2018-06-29 19:18:08+00:00,5876499,0x8a53a593563d70e174bf711f44d1474f5f2b1fb3be7e197854754cf758f5a99f,missing,missing,missing,15000000000


In [141]:
@time results = apply_linked_transactions_heuristic(transactions_of_deposit_addresses_compacted, withdraw_transactions_df)

0.0%┣                                     ┫ 0/83.8k [00:30<-698:-11:00, -30s/it]
0.0%┣                                        ┫ 1/83.8k [00:30<Inf:Inf, InfGs/it]
13.2%┣█████                                 ┫ 11.0k/83.8k [01:00<06:36, 184it/s]
18.9%┣███████▏                              ┫ 15.8k/83.8k [01:30<06:27, 176it/s]
23.8%┣█████████                             ┫ 20.0k/83.8k [02:00<06:24, 166it/s]
27.9%┣██████████▋                           ┫ 23.3k/83.8k [02:30<06:28, 156it/s]
31.2%┣███████████▉                          ┫ 26.2k/83.8k [03:00<06:36, 145it/s]
34.2%┣█████████████                         ┫ 28.7k/83.8k [03:30<06:44, 137it/s]
37.0%┣██████████████                        ┫ 31.0k/83.8k [04:00<06:49, 129it/s]
39.5%┣███████████████                       ┫ 33.1k/83.8k [04:30<06:54, 123it/s]
42.0%┣████████████████                      ┫ 35.2k/83.8k [05:00<06:54, 117it/s]
44.4%┣████████████████▉                     ┫ 37.2k/83.8k [05:30<06:54, 113it/s]
46.6%┣█████████████████▊    

,withdraw_address,deposit_address,transaction_hash
,String,String,String
1,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x55e2780588aa5000f464f700d2676fd0a22ee160,0xa5bd1dffc78fa0d1d49b0f66fb069d30a292c0f4090196e4a7ed5db1cb27b7a9
2,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x5fc94341ba178e4ca1e2069baeb8e3fd1201cbb0,0xd37b2a60d6de81bd58eca811fb511d8bbed30f3d8a62b6eb6cb505585f1cd2e1
3,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x7471f5cbacb71968bf5816d19a4bf3f6bd863b88b6ad467e450382a60ff5f72e
4,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x5fc94341ba178e4ca1e2069baeb8e3fd1201cbb0,0x95aa6281da2b63b8fc0f0a11b13ce42f961ad3c131ca93109d538853a0914747
5,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x4a6f9846fbbde38fe51147b0e6c274a607e2b7b0,0x379f0696a50123ac7917943815b72540159b737df8ac76e0c9ed3f4e8b06228a
6,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x2ea70aea444d26597eca601d210d2b9ce2cdf999627afa6bd205d5792df97c64
7,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x1a2952c9bc0177e5c40a22cfa00c9f0d0e2070d6,0x4ed59293c3b882377a23cd4ecd2f3859012c4bcd882d5c13962fa8157e51437d
8,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0xa43ce8cc89eff3aa5593c742fc56a30ef2427cb0,0x00fd2e97edefc06d06713ea49e7a1d22ff87a4875e3d27888387ab10e9d0c388
9,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x4a6f9846fbbde38fe51147b0e6c274a607e2b7b0,0x85f1ed3f7a7eece62a83c430a34c420a123ffd3126afa833a46cc345e037568c


Lets inspect the results

In [177]:
groupby_results = sort(combine(groupby(results, [:withdraw_address, :deposit_address]), nrow => :count), :count, rev = true)

,withdraw_address,deposit_address,count
,String,String,Int64
1,0x9dfac9bbe1119da5dcdd7b095db40920cde878ef,0xe5818d70a9b5aed2bfde4e41fbcb07dd80f8fc84,343
2,0xa8015df1f65e1f53d491dc1ed35013031ad25034,0x8298c072baa440607c1f83b4024c0b4c47fcd163,231
3,0x01349510117dc9081937794939552463f5616dfb,0xb435871b0959561226b4d903b1abf79528177e81,140
4,0xf343baf675e990baf450d08fa05bd9799de328d2,0x510298b956059b15f2028887b757d53cdff55ce7,129
5,0x9521f77aa4f5f3dd5c4b2da897c13b0c3d4e9791,0x7eff673debd328955e4705ff3ce99fb0f3789d0d,114
6,0x3d475025275ffb03a7fff9adb61acb93cf9122ef,0x9521f77aa4f5f3dd5c4b2da897c13b0c3d4e9791,110
7,0x97b65ad59c8c96f2dd786751e6279a1a6d34a481,0xac6342a7efb995d63cc91db49f6023e95873d250,87
8,0xa8015df1f65e1f53d491dc1ed35013031ad25034,0x77144cb43e4e8ae1c351196c6513813c44b21f99,78
9,0x99c515bc95e43697b38f419d5359a3876b68e538,0xbbbfc46566e5f0302cef913af8c8f423070ce6a1,75


In [ ]:
CSV.write("../data/heuristic_3_groupby_results.csv", get_linked_addresses(groupby_results))

In [171]:
sort(x, :withdraw_address)

,withdraw_address,deposit_address,count
,String,String,Int64
1,0x00000000009a41862f3b2b0c688b7c0d1940511e,0x185a38208179db734f43c704062b26793e3263cd,3
2,0x00000000009a41862f3b2b0c688b7c0d1940511e,0xe747c2d98e21de4f170fb65e84e8fa2f840c6e1c,1
3,0x0000000000bc14115f9f67fde839f285667437bc,0x0000000000bc14115f9f67fde839f285667437bc,1
4,0x0000000000bc14115f9f67fde839f285667437bc,0x7bb8536bac7d6e5c4e352ce50be8968d5d6cd445,1
5,0x000000009466503fd80b5cf8cb539421a1c7f41b,0xc3aae58ab81663872dd36d73613eb295b167f546,10
6,0x00000000aad2f2cde34c182a8e03fc75e6626b7d,0xf7980c30831f96334bf27832141ae062499b540e,1
7,0x00000081c22fe36e0b47a4f1c67041301f945014,0x530c742ddfcfb76ae7621e9f7717768e0c0cca7a,7
8,0x00000081c22fe36e0b47a4f1c67041301f945014,0xeaa618ac79eb74f5bf5cff98dab046053ffb1e99,2
9,0x00002b503a75998c97508916a74fdb41934fa030,0x520e74218a9fd5563855f11d204810281a833e0f,6


As we can see some withdraw_address are linked to more than one deposit_address and viceversa.
We will define that a `withdraw_address`and a `deposit_address` are linked, in the cases that the `withdraw_address` is only linked to the `deposit_address` and the `deposit_address` is only linked to the `withdraw_address

In [282]:
# Auxiliary functions check if the given address is unique in the input column
function is_unique(address, column)
    return findall(row -> row == address, column ) |> length == 1 
end

is_unique (generic function with 1 method)

In [237]:
function get_linked_addresses(results)
    grouped_results = combine(groupby(results, [:withdraw_address, :deposit_address]), nrow => :count)
    withdraw_addresses = grouped_results[!, :withdraw_address]
    deposit_addresses = grouped_results[!, :deposit_address]
    linked_addresses = Dict()
    for row ∈ eachrow(grouped_results)
        if is_unique(row.withdraw_address, withdraw_addresses) && is_unique(row.deposit_address, deposit_addresses)
            linked_addresses[row.withdraw_address] = row.deposit_address
        end
    end
    return linked_addresses
end

get_linked_addresses (generic function with 1 method)

In [222]:
get_linked_addresses(results)

0.0%┣                                      ┫ 0/5.0k [00:30<-41:-18:-30, -30s/it]
0.0%┣                                         ┫ 1/5.0k [00:30<Inf:Inf, InfGs/it]
100.0%┣███████████████████████████████████████┫ 5.0k/5.0k [00:30<00:00, 163it/s]


Dict{Any, Any} with 552 entries:
  "0x50892e106095f415b4b121652a9808fe52664f82" => "0xcd79b477f7d66f35b603aa44a7d7ca03e987077d"
  "0x6b91f0343b2160514cb3058282082583956beb22" => "0x16e54b35d789832440ab47ae765e6a8098280676"
  "0xb0ec510be0cd0bfcff595955be598ca0b3b8ec4e" => "0xb0ec510be0cd0bfcff595955be598ca0b3b8ec4e"
  "0xc0ffee22f360792a89db39f159f103daafe9c57d" => "0x75d4bdbf6593ed463e9625694272a0ff9a6d346f"
  "0x57fdb9005cba66aa26e5882eb87850d80a9a188b" => "0x5550519d3ded948884ea0337e3524c24955115d2"
  ⋮                                            => ⋮

In [ ]:
CSV.write("../data/heuristic_3_linked_addresses.csv", get_linked_addresses(results))

### Results visualization

In [241]:
function get_histogram_data(results, withdraw_transactions_df)
    grouped_results = combine(groupby(results, [:withdraw_address, :deposit_address]), nrow => :count)
    withdraw_addresses = grouped_results[!, :withdraw_address]
    deposit_addresses = grouped_results[!, :deposit_address]
    linked_addresses = Dict()
    
    for withdraw_address in withdraw_transactions_df[!, :recipient_address] |> unique
        linked_addresses[withdraw_address] = 0
    end
    
    for row ∈ eachrow(grouped_results)
        linked_addresses[row.withdraw_address] += 1
    end
    
    return linked_addresses
end

get_histogram_data (generic function with 2 methods)

In [242]:
get_histogram_data(results, withdraw_transactions_df)

Dict{Any, Any} with 32528 entries:
  "0xab17da946b4ee971e6cd95f9784d664c074c1eae" => 0
  "0x3db8a6a96a8e3711c30c84d5bc26971ef61c549d" => 0
  "0x5d57f2e5f61b484eadc146fe245ff67cceb24497" => 0
  "0xce41f28db174e9684c84a1edcfddbebeaa22729d" => 0
  "0xe8c4d0b45cae962ea5fab9f797f49124c7e2d8ca" => 0
  ⋮                                            => ⋮

In [288]:
result_values = values(get_histogram_data(results, withdraw_transactions_df)) |> collect
histogram(result_values, bins= maximum(result_values)+1 ,legend=false, title="Amoun of deposit addresses linked with the withdraw transactions", xlabel="Withdraws")

LoadError: UndefVarError: results not defined

In [289]:
#Lets see only the withdraws that are linked to a deposit address
result_values = values(get_histogram_data(results, withdraw_transactions_df)) |> collect |> sort 
idx = findlast(score -> score == 0, result_values)
histogram(result_values[idx:length(result_values)], bins= maximum(result_values) + 1, legend=false,  title="Amoun of deposit addresses linked with the withdraw transactions", xlabel="Withdraws")

LoadError: UndefVarError: results not defined